In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#required packages.
import numpy as np
import pandas as pd
from pandas import Timestamp
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
import datetime
import itertools
import argparse
import collections

Prior to using the Earth Engine Python client library, you need to authenticate (verify your identity) and use the resultant credentials to initialize the Python client. The authentication flows use Cloud Projects to authenticate, and they're used for unpaid (free, noncommercial) use as well as paid use. To authenticate and initialize, run the following codes.

https://developers.google.com/earth-engine/guides/auth


In [ ]:
import ee
# # Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='')

## utils

In [ ]:
#must run these codes
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

def ee_array_to_df1(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors="coerce")

    # Convert the time field into a datetime.
    df["datetime"] = pd.to_datetime(df["time"], unit="ms")

    # Keep the columns of interest.
    df = df[["time", "datetime", *list_of_bands]]

    # The datetime column is defined as index.
    df = df.set_index("datetime")

    return df

## MODIS vegetation condition
* MOD17A3HGF.061: Terra Net Primary Production Gap-Filled Yearly Global 500m https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD17A3HGF
* MCD64A1.006 MODIS Burned Area Monthly Global 500m https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD64A1#bands
* WorldClim BIO Variables https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO
* MOD13Q1.006 Terra Vegetation Indices 16-Day Global 250m https://developers.google.com/earth-engine/datasets/catalog/modis
* MCD43A4.006 MODIS Nadir BRDF-Adjusted Reflectance Daily 500m https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD43A4
* MOD11A2.006 Terra Land Surface Temperature and Emissivity 8-Day Global 1km
https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A2



In [ ]:
def get_modis_fpar_lai(r_lon, r_lat, scale, i_date, f_date):
    # Import soil temperature.
    selected_features = ['Fpar', 'Lai']
    pr = (ee.ImageCollection('MODIS/006/MCD15A3H')
        .select(selected_features)
        .filterDate(i_date, f_date))

    # Define the location of interest
    poi = ee.Geometry.Point(r_lon, r_lat)
    # Evaluate local precipitation conditions.
    local_pr = pr.getRegion(poi, scale).getInfo()
    pr_df = ee_array_to_df(local_pr, selected_features)

    return pr_df

def get_modis_statistic_features(prd):
    dfcd = prd.describe()
    dfc_mean = dfcd.loc[['mean']].reset_index(drop=True)
    dfc_std = dfcd.loc[['std']].reset_index(drop=True)
    dfc_25p = dfcd.loc[['25%']].reset_index(drop=True)
    dfc_50p = dfcd.loc[['50%']].reset_index(drop=True)
    dfc_75p = dfcd.loc[['75%']].reset_index(drop=True)

    dfc_mean.rename(update_dictionary_mean, axis=1, inplace=True)
    dfc_std.rename(update_dictionary_std, axis=1, inplace=True)
    dfc_25p.rename(update_dictionary_25p, axis=1, inplace=True)
    dfc_50p.rename(update_dictionary_50p, axis=1, inplace=True)
    dfc_75p.rename(update_dictionary_75p, axis=1, inplace=True)

    dfs2 = pd.concat([dfc_25p, dfc_50p, dfc_75p, dfc_mean, dfc_std], axis=1)

    return dfs2

In [ ]:
#collect MODIS features
scale = 100  # scale in meters
#interested time period
i_date, f_date = '2018-01-01', '2018-12-31'
r_lon, r_lat = 46.857320,13.893164

get_modis_fpar_lai(r_lon, r_lat, scale, i_date, f_date)

,time,datetime,Fpar,Lai


## MODIS time serie images

In [ ]:
def get_modis_time_series_image(aoi_modis, target_band, i_date, f_date):
    # Load a collection
    TERRA = (ee.ImageCollection("MODIS/006/MOD11A1")
              .filterDate(i_date, f_date)
              .filterBounds(aoi_modis)  # Filter images containing AOI
              .sort('system:time_start'))

    # Extract an image
    TERRA_list = TERRA.toList(TERRA.size())

    numpy_array = []
    for i in range(TERRA.size().getInfo()):
        Terra_img = ee.Image(TERRA_list.get(i))

        # Use reduceRegion to get a dictionary of mean values in the ROI.
        mean_dict = Terra_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi_modis, scale=1000)

        # The dictionary contains a mean value for each band in the image. Retrieve the value for your band.
        mean_val = mean_dict.get(target_band).getInfo()

        if mean_val:  # If the mean value exists (is not None), append it to the list.
            numpy_array.append(mean_val)
        else:
            print(f"Image {i} does not contain the AOI or is fully masked")
    return numpy_array

def get_modis_time_series_image_new(aoi_modis, target_band, i_date, f_date, scale):
    # Load a collection
    TERRA = (ee.ImageCollection("MODIS/006/MOD11A1")
              .select([target_band])
              .filterDate(i_date, f_date)
              .filterBounds(aoi_modis)  # Add this line to filter images containing AOI
              .sort('system:time_start'))

    # Extract an image
    TERRA_list = TERRA.toList(TERRA.size())

    numpy_array = []
    for i in range(TERRA.size().getInfo()):
        Terra_img = ee.Image(TERRA_list.get(i))
        #reproject image with user defined scale
        Terra_img = Terra_img.reproject(crs = ee.Projection('EPSG:4326'), scale=scale)

        # check if image footprint contains the AOI
        image_footprint = Terra_img.geometry()
        if image_footprint.contains(aoi_modis).getInfo():
            sample = Terra_img.select(target_band).sampleRectangle(region=aoi_modis, defaultValue=0)
            numpy_array_new = np.array(sample.get(target_band).getInfo())
            numpy_array.append(numpy_array_new)
        else:
            print(f"Image {i} does not contain the AOI")
    numpy_array = np.array(numpy_array)
    return numpy_array

In [ ]:
#input parameter
aoi = ee.Geometry.Polygon(
        [[[1.665586676010049, 48.55860495478288],
          [1.665586676010049, 48.536502067708305],
          [1.7085878509368069, 48.536502067708305],
          [1.7085878509368069, 48.55860495478288]]])
target_band = 'LST_Day_1km'
i_date, f_date = '2017-12-01', '2019-01-31'
scale = 10
ImgTS_total_day = get_modis_time_series_image_new(aoi, target_band, i_date, f_date, scale)
ImgTS_total_day.shape